In [1]:
import requests
import time
import pandas as pd
from datetime import date, datetime

In [2]:
spark.conf.set(
  "fs.azure.account.key.adfandreiablueshift.blob.core.windows.net",
  "GHFhO6sVusVBxM4obc4GIqRufA+6mrYy/4PlBXsVgEnbkPuZ4vQ0sN09afisz/2fpDzh+NBfz6A16PYEjyvXHw==")

output_path = "wasbs://projetointegrado@adfandreiablueshift.blob.core.windows.net//ApiTeste"

In [3]:
data = datetime.now() #identifica a data e hora do start.
anomes = int(data.strftime('%Y%m')) #transforma a data em ano/mês em número inteiro (formato que o arquivo chega).
codibge=3500105 
dtibge=202003 #primeiro mês de referência.
res2 = []

#Identificar os IDs dos municipios, de acordo com o site do IBGE.
res = []
u='https://servicodados.ibge.gov.br/api/v1/localidades/distritos' #lista de códigos de município, para ser usado dentro do r.
r=requests.get(u) #chama a u(url)
d=pd.read_json(r.content.decode('utf-8')) #lê o arquivo que voltar da solicitação do r, e salvar em um DF. 
x=r.json() #Lê os dados json (transforma em dicinário).
res=pd.json_normalize(x) #insere os dados json dentro do DF. O Normalize serve para normalizar os dados, por exemplo, se tiver subarquivos, ele nivela.
id_ibge=res['municipio.id'] #pega a coluna município.id e insere em uma variável. 
id_ibge=['520005005', '310010405']



2020-09-10 14:53:43.714691

In [4]:
#Buscar as informações do bolsa família.
num = 0
iccsv=0
while dtibge <= anomes: #enquanto o dtibge for menor igual a data atual.
    for i in id_ibge: #para cada item do id ibge
        if num == 30: #se num = 30
            print('Aguardando....')
            time.sleep(60) #aguardar 60s
            num = 0 #num volta para 0
            
        num = num+1 #começa a rodar de novo
        headers = {
        'User-Agent': 'request',
        'chave-api-dados': '4d85f7d1d3dfca2ccc0602434ad67c93',
        } #credenciais fornecidas pela api
        url = "http://www.portaltransparencia.gov.br/api-de-dados/bolsa-familia-por-municipio?mesAno="+str(dtibge)+"&codigoIbge="+str(i)+"&pagina=1" #i = item da lista
        #print(url)
        try:
            response = requests.get(url, headers=headers)
            df=pd.read_json(response.content.decode('utf-8')) #transforma o arquivo recebido em DF.
            d = response.json()
            d = r
            df = pd.json_normalize(r) 
            for i, c in df.iterrows(): #obtém o valor da próxima linha.
              res2.append(c)
       

In [5]:
 except:
            print('erro: '+url)
        if iccsv==0: #se o iccsv for 0 o arquivo nao tem a primeira linha (cabecalho), ele vai inserir o cabecalho e salvar em uma pasta.
            df.to_csv('wasbs://projetointegrado@adfandreiablueshift.blob.core.windows.net//Bolsa Familia/API_Data/Teste_API', sep=';', encoding='utf-8', header='true', index=False)
            iccsv ==1 #If roda apenas a primeira vez. 
        else:
          df.to_csv('wasbs://projetointegrado@adfandreiablueshift.blob.core.windows.net//Bolsa Familia/API_Data/Teste_API', sep=';', encoding='utf-8', header=False, index=False, mode='a')
          print('check')
          dtibge=dtibge+1 #quando terminar, ele vai para o proximo mês.